In [5]:
# special big thanks to the library for figuring this out!

import pdfplumber
import pandas as pd
import os.path as path

In [6]:
def clean_data(df):
    return (df
        .replace(r'^s*$', float('NaN'), regex = True)  # There were blank rows, so first I changed them to "NaN" values
        .dropna(axis=0, how='all')  # Then dropped any row where all the values were NaN
        .assign(street=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.rsplit('#', n=1, expand=True)[0].str.rsplit(',', expand=True)[0], # Get street data. Assumes # or comma or nothing
                unit1=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.extract(r'(\d+$)|(\#\w+$)')[1].str.strip('#'), # Get unit data
                unit2=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.rsplit('#', n=1, expand=True)[1], # Get unit data
                unit3=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('unit', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                unit4=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('apt', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                unit5=df.Address.str.rsplit('\n', n=1, expand=True)[0].str.lower().str.rsplit('suite', n=1, expand=True)[1].str.strip().str.lstrip('#'), # Get unit data
                city=df.Address.str.rsplit('\n', n=1, expand=True)[1].str.split(',', n=1, expand=True)[0],
                state=df.Address.str.rsplit('\n', n=1, expand=True)[1].str.split(',', n=1, expand=True)[1].str.rsplit(' ', n=1, expand=True)[0].str.strip(),
                zip=df.Address.str.rsplit('\n', n=1, expand=True)[1].str.split(',', n=1, expand=True)[1].str.rsplit(' ', n=1, expand=True)[1].str.strip())
    )

def load_within_directory(directory_string):
    """
    Takes in a string of where your csv is located in the repo folder and turns
    into a dataframe using pandas read_csv

    example directory string: '/data/raw_data/CIE/client_needs_table.csv'
    :param directory_string: string containing file with directory desired
    :return: a dataframe
    """

    temp_path = path.abspath(path.join("data" ,"../../..")) # finds the parent directory

    # concatenates with directory
    return temp_path + directory_string


In [7]:
pdf = pdfplumber.open(load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/sdso_lockout_service_activity_details_jan_2018_jan_2023.pdf'))

table_settings = {
    "explicit_vertical_lines": [20, 100, 220, 370, 465, 525, 600],
    "explicit_horizontal_lines": [761],
    "horizontal_strategy": "lines",
    "intersection_x_tolerance": 50,
    "join_tolerance": 300
}

# Scrape PDF and put rows into a list
row_data = []
for x in range(len(pdf.pages)):
    p = pdf.pages[x]
    table = p.extract_table(table_settings)
    for row in table:
        row_data.append(row)

# Create a pandas dataframe from the pdf data being stored in a list
df = pd.DataFrame(row_data, columns=['File_Number', 'Occupants', 'Address', 'Restoration_Date', 'Time', 'Status'])

# Clean the raw data from the pdf
df2 = clean_data(df)

# Combine all the unit columns into one
df2.unit1.update(df2.unit2)
df2.unit1.update(df2.unit3)
df2.unit1.update(df2.unit4)
df2.unit1.update(df2.unit5)

In [10]:
directory = load_within_directory('/data/raw_data/sheriff_evictions_2018_jan_2023/')
directory = directory + 'evictions_library_export.csv'
df2.to_csv(directory, index=False)

In [9]:
# (df2
#     .drop(columns=['unit2', 'unit3', 'unit4', 'unit5']) # Drop the extra unit columns since they're now combined
#     .rename(columns={'unit1': 'unit'}) # Rename the combined unit1 column to unit
#     .to_excel('evictions_cleaned.xlsx', index=False) # Save to Excel
# )
